# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

**Roseman Sales Dataset** - This dataset is a live dataset of Roseman Stores. On analsysing this problem we observe that Roseman problem is a regression problem and our primarily goal is to predict the sales figures of Roseman problem. In this Notebook we work on following topics

Analysing the Dataset by using Exploratory Data Analysis.
Using Exponential Moving Averages analyse Trends and Seasonality in Roseman dataset.
Analyse Regression analysis using following prediction analysis, A. Linear Regression Analysis B. Elastic Regression ( Lasso and Ridge Regression). C. Random Forest Regression. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
import math
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#filepath='/content/drive/MyDrive/retail sales regression/csv files/'
store_df=pd.read_csv('C:/Users\Rahul\Desktop\Data_Science\AlmaBetter\Machine Learning\Linear Regression\Project\Regression/store.csv')
rossman_df=pd.read_csv("C:/Users\Rahul\Desktop\Data_Science\AlmaBetter\Machine Learning\Linear Regression\Project\Regression/Rossmann Stores Data.csv")

I did some basic first hand explorations by looking at data frames,looking at its shape,info etc.

In [4]:
store_df.head()


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
rossman_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [6]:
rossman_df.shape,store_df.shape

((1017209, 9), (1115, 10))

**DATA PREPROCESSING**   
I looked for nan values in store data.there were a lot of nan values in some columns.the had to be dealt with.So i wrote a code to specifically deal with the nan values of each column either by replacing it with 0,mode or median.Removing the columns was not an option as they might remove some significant amount of data.

In [7]:
store_df.isna().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [8]:
store_new = store_df.copy()
store_new['CompetitionDistance'] = store_new['CompetitionDistance'].fillna(0)
store_new['CompetitionOpenSinceMonth'] = store_new['CompetitionOpenSinceMonth'].fillna(0)
competition_year_mode = store_new['CompetitionOpenSinceYear'].mode().iloc[0]
store_new['CompetitionOpenSinceYear'] = store_new['CompetitionOpenSinceYear'].fillna(competition_year_mode)


store_new['Promo2SinceWeek'] = store_new['Promo2SinceWeek'].fillna(0)
promo2year_mode = store_new['Promo2SinceYear'].mode().iloc[0]
store_new['Promo2SinceYear'] = store_new['Promo2SinceYear'].fillna(promo2year_mode)
promo_interval_mode = store_new['PromoInterval'].mode().iloc[0]
store_new['PromoInterval'] = store_new['PromoInterval'].fillna(promo_interval_mode)
store_new.isna().sum()
#Mapping everything to numerical values
store_new['StoreType']    = store_new['StoreType'].map({'a':1,'b':2,'c':3,'d':4})
store_new['Assortment']   = store_new['Assortment'].map({'a':0, 'c':1,'b':2})
store_new['PromoInterval']= store_new['PromoInterval'].map({'Jan,Apr,Jul,Oct':0, 'Feb,May,Aug,Nov':1,'Mar,Jun,Sept,Dec':2})
store_new.info()
store_new.StoreType.unique()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   int64  
 2   Assortment                 1115 non-null   int64  
 3   CompetitionDistance        1115 non-null   float64
 4   CompetitionOpenSinceMonth  1115 non-null   float64
 5   CompetitionOpenSinceYear   1115 non-null   float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            1115 non-null   float64
 8   Promo2SinceYear            1115 non-null   float64
 9   PromoInterval              1115 non-null   int64  
dtypes: float64(5), int64(5)
memory usage: 87.2 KB


array([3, 1, 4, 2], dtype=int64)

In [9]:
store_new.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,3,0,1270.0,9.0,2008.0,0,0.0,2011.0,0
1,2,1,0,570.0,11.0,2007.0,1,13.0,2010.0,0
2,3,1,0,14130.0,12.0,2006.0,1,14.0,2011.0,0
3,4,3,1,620.0,9.0,2009.0,0,0.0,2011.0,0
4,5,1,0,29910.0,4.0,2015.0,0,0.0,2011.0,0


**MERGING THE DATAFRAMES** now as the nan values were removed i decided to merge the two dataframes with left join on rossman_df,as the rossman data had more columns.

In [10]:
final1 = pd.merge(left=rossman_df, right=store_new,
                               how='left', on=['Store'])
final1.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,3,0,1270.0,9.0,2008.0,0,0.0,2011.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,1,0,570.0,11.0,2007.0,1,13.0,2010.0,0
2,3,5,2015-07-31,8314,821,1,1,0,1,1,0,14130.0,12.0,2006.0,1,14.0,2011.0,0
3,4,5,2015-07-31,13995,1498,1,1,0,1,3,1,620.0,9.0,2009.0,0,0.0,2011.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,1,0,29910.0,4.0,2015.0,0,0.0,2011.0,0


In [11]:
final1.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01,2.207467e+00,4.798011e-01,5.415982e+03,4.926878e+00,2.010060e+03,5.005638e-01,1.164767e+01,2.011377e+03,3.082120e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01,1.365376e+00,5.156546e-01,7.710253e+03,4.283543e+00,5.340757e+00,4.999999e-01,1.532393e+01,1.235230e+00,6.362705e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.900000e+03,0.000000e+00,0.000000e+00,2.009000e+03,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,7.000000e+02,0.000000e+00,2.008000e+03,0.000000e+00,0.000000e+00,2.011000e+03,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.320000e+03,4.000000e+00,2.012000e+03,1.000000e+00,1.000000e+00,2.011000e+03,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00,4.000000e+00,1.000000e+00,6.880000e+03,9.000000e+00,2.013000e+03,1.000000e+00,2.200000e+01,2.012000e+03,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,2.000000e+00,7.586000e+04,1.200000e+01,2.015000e+03,1.000000e+00,5.000000e+01,2.015000e+03,2.000000e+00


In [12]:
final1.info

<bound method DataFrame.info of          Store  DayOfWeek        Date  Sales  Customers  Open  Promo  \
0            1          5  2015-07-31   5263        555     1      1   
1            2          5  2015-07-31   6064        625     1      1   
2            3          5  2015-07-31   8314        821     1      1   
3            4          5  2015-07-31  13995       1498     1      1   
4            5          5  2015-07-31   4822        559     1      1   
...        ...        ...         ...    ...        ...   ...    ...   
1017204   1111          2  2013-01-01      0          0     0      0   
1017205   1112          2  2013-01-01      0          0     0      0   
1017206   1113          2  2013-01-01      0          0     0      0   
1017207   1114          2  2013-01-01      0          0     0      0   
1017208   1115          2  2013-01-01      0          0     0      0   

        StateHoliday  SchoolHoliday  StoreType  Assortment  \
0                  0              1      

Now i had a data frame named final1.I saw that the date column is not much helping here to i extracted month from it and dropped the columns.

In [13]:
final1['Date'] = pd.to_datetime(final1['Date'])
final1['sales_month'] = final1['Date'].dt.month
cols_drop = ['Date', 'Customers', 'Store']
final1['StateHoliday'] = final1['StateHoliday'].replace({0: '0'})
final1 = final1.drop(cols_drop, axis=1)
final1_dummies = pd.get_dummies(final1, drop_first=True)

In [14]:
final1['Sales'].mean()

5773.818972305593

In [15]:
final1_dummies.shape

(1017209, 18)

In [16]:
final1_dummies.head()

,DayOfWeek,Sales,Open,Promo,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,sales_month,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,5,5263,1,1,1,3,0,1270.0,9.0,2008.0,0,0.0,2011.0,0,7,0,0,0
1,5,6064,1,1,1,1,0,570.0,11.0,2007.0,1,13.0,2010.0,0,7,0,0,0
2,5,8314,1,1,1,1,0,14130.0,12.0,2006.0,1,14.0,2011.0,0,7,0,0,0
3,5,13995,1,1,1,3,1,620.0,9.0,2009.0,0,0.0,2011.0,0,7,0,0,0
4,5,4822,1,1,1,1,0,29910.0,4.0,2015.0,0,0.0,2011.0,0,7,0,0,0


**Modelling**:   
These are the columns that i kept for modelling
['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'sales_month', 'StateHoliday_a',
       'StateHoliday_b', 'StateHoliday_c'].Ill be using LinearRegression,decision trees,random forest,LightGBM regressor etc

In [17]:
dep_var='Sales'
indep_var=final1_dummies.columns.drop(dep_var)

In [18]:
indep_var

Index(['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'sales_month', 'StateHoliday_a',
       'StateHoliday_b', 'StateHoliday_c'],
      dtype='object')

In [19]:
X=final1_dummies[indep_var]
y=final1_dummies[dep_var]


In [20]:
X

,DayOfWeek,Open,Promo,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,sales_month,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,5,1,1,1,3,0,1270.0,9.0,2008.0,0,0.0,2011.0,0,7,0,0,0
1,5,1,1,1,1,0,570.0,11.0,2007.0,1,13.0,2010.0,0,7,0,0,0
2,5,1,1,1,1,0,14130.0,12.0,2006.0,1,14.0,2011.0,0,7,0,0,0
3,5,1,1,1,3,1,620.0,9.0,2009.0,0,0.0,2011.0,0,7,0,0,0
4,5,1,1,1,1,0,29910.0,4.0,2015.0,0,0.0,2011.0,0,7,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,2,0,0,1,1,0,1900.0,6.0,2014.0,1,31.0,2013.0,0,1,1,0,0
1017205,2,0,0,1,3,1,1880.0,4.0,2006.0,0,0.0,2011.0,0,1,1,0,0
1017206,2,0,0,1,1,1,9260.0,0.0,2013.0,0,0.0,2011.0,0,1,1,0,0
1017207,2,0,0,1,1,1,870.0,0.0,2013.0,0,0.0,2011.0,0,1,1,0,0


In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [22]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((813767, 17), (203442, 17), (813767,), (203442,))

In [23]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [24]:
dt_regressor=DecisionTreeRegressor(max_depth=5)
dt_regressor.fit(X_train, y_train)
y_pred=dt_regressor.predict(X_test)
y_train_pred=dt_regressor.predict(X_train)
print('dt_regressor R^2: ', r2(y_test,y_pred))
print('dt_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))
print('dt_regressor RMSE: ', math.sqrt(mse(y_train, y_train_pred)))
#print('dt_regressor RMSE: ',(np.sqrt(np.mean(np.square((y_test-y_pred)/y_test))))*100

dt_regressor R^2:  0.5796454472107977
dt_regressor RMSE:  2502.5624536144182
dt_regressor RMSE:  2487.621136689337


In [ ]:
rf_regressor=RandomForestRegressor(n_estimators =500,max_depth=5)
rf_regressor.fit(X_train, y_train)
y_pred=rf_regressor.predict(X_test)
print('rf_regressor R^2: ', r2(y_test,y_pred))
print('rf_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))

In [ ]:
reg=LinearRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)
print('R^2: ', r2(y_test,y_pred))
print('RMSE: ',math.sqrt(mse(y_test,y_pred)))
